In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import pandas_ta as ta
from scipy.ndimage import gaussian_filter
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, clear_output
import mplfinance as mpf

PORTFOLIO_SIZE = 100000

In [2]:
# Function to calculate technical indicators
def calculate_all_indicators(stock_data, selected_date):
    """
    Calculate all technical indicators including SMAs, MACD Histogram, ATR, and candlestick patterns for the selected date.
    
    Args:
    - stock_data (pd.DataFrame): Stock data containing 'Close', 'High', 'Low', etc.
    - selected_date (str): The date for which the technical indicators should be calculated.
    
    Returns:
    - A dictionary with all calculated indicators.
    """
    selected_date = pd.to_datetime(selected_date)

    # Calculate SMAs
    stock_data['SMA_200'] = ta.sma(stock_data['Close'], length=200)
    stock_data['SMA_50'] = ta.sma(stock_data['Close'], length=50)
    stock_data['SMA_20'] = ta.sma(stock_data['Close'], length=20)

    # Calculate MACD Histogram
    stock_data['MACD_Histogram'] = ta.macd(stock_data['Close'])['MACDh_12_26_9']

    # Calculate ATR
    stock_data['ATR_14'] = ta.atr(stock_data['High'], stock_data['Low'], stock_data['Close'], length=14)

    # Calculate candlestick patterns for the selected date
    candlestick_patterns = stock_data.ta.cdl_pattern(name="all").loc[selected_date]

    # Get the row for the selected date to extract indicator values
    data_for_date = stock_data.loc[selected_date]

    return {
        'SMA_200': data_for_date['SMA_200'],
        'SMA_50': data_for_date['SMA_50'],
        'SMA_20': data_for_date['SMA_20'],
        'MACD_Histogram': data_for_date['MACD_Histogram'],
        'ATR_14': data_for_date['ATR_14'],
        'Candlestick_Patterns': candlestick_patterns
    }

# Risk-Reward Calculation Function
def calculate_risk_reward(current_price, stop_loss, main_resistance):
    """Calculate risk, reward, risk-reward ratio, and suggested limit price."""
    if stop_loss and main_resistance:
        risk = current_price - stop_loss
        reward = main_resistance - current_price
        risk_reward_ratio = reward / risk if risk > 0 else None
        
        # Estimated gain as a percentage
        estimated_gain = (main_resistance - current_price) / current_price * 100 if current_price > 0 else None
        
        # Suggested limit price using (main_resistance + 2 * stop_loss) / 3
        suggested_limit_price = (main_resistance + 2 * stop_loss) / 3 if stop_loss is not None and main_resistance is not None else None
        
        # Ensure the limit price is within the main support and resistance
        if suggested_limit_price <= stop_loss or suggested_limit_price >= main_resistance:
            suggested_limit_price = None
    else:
        risk_reward_ratio = None
        estimated_gain = None
        suggested_limit_price = None
    
    return risk_reward_ratio, estimated_gain, suggested_limit_price

# Function to format and sort candlestick patterns for readability
def format_candlestick_patterns(patterns_dict):
    """
    Formats the candlestick patterns dictionary into a comma-separated string, sorted by value in descending order.
    
    Args:
    - patterns_dict (dict): The candlestick patterns dictionary.
    
    Returns:
    - str: A formatted string of sorted candlestick patterns or an empty string if no patterns.
    """
    if not patterns_dict:
        return ""

    # Filter patterns with non-zero values and sort them by value (descending order)
    sorted_patterns = sorted(((pattern, value) for pattern, value in patterns_dict.items() if value != 0),
                             key=lambda x: x[1], reverse=True)

    # Format the sorted patterns
    formatted_patterns = [f"{pattern}: {value}" for pattern, value in sorted_patterns]

    # Return a comma-separated string
    return ", ".join(formatted_patterns)

# Helper function to compute the main support with fallback logic
def compute_main_support(volume_profile_df, current_price, atr_value):
    """
    Computes the main support, selecting the highest scoring support if no main support is found.

    Args:
    - volume_profile_df (pd.DataFrame): DataFrame containing support and resistance levels.
    - current_price (float): The current price of the stock.
    - atr_value (float): The ATR value used to calculate the stop loss.

    Returns:
    - (main_support, main_support_score, stop_loss): The main support price, its score, and the stop loss.
    """
    # Get the main support row
    main_support_row = volume_profile_df[volume_profile_df['Label'] == 'Main Support']

    if not main_support_row.empty:
        main_support = main_support_row.iloc[0]['Price']
        main_support_score = main_support_row.iloc[0]['Score']
        stop_loss = main_support - atr_value  # Risk: 1 ATR below main support
    else:
        # Fallback: find the highest scoring support below the current price
        fallback_support_row = volume_profile_df[
            (volume_profile_df['Price'] < current_price) & (volume_profile_df['Score'] > 0)
        ].sort_values(by='Score', ascending=False).head(1)

        if not fallback_support_row.empty:
            main_support = fallback_support_row.iloc[0]['Price']
            main_support_score = fallback_support_row.iloc[0]['Score']
            stop_loss = main_support - atr_value  # Risk: 1 ATR below fallback support
        else:
            main_support = None
            main_support_score = None
            stop_loss = None

    return main_support, main_support_score, stop_loss


# Helper function to compute the main resistance with fallback logic
def compute_main_resistance(volume_profile_df, current_price):
    """
    Computes the main resistance, selecting the closest non-zero score resistance line if no main resistance is found.

    Args:
    - volume_profile_df (pd.DataFrame): DataFrame containing support and resistance levels.
    - current_price (float): The current price of the stock.

    Returns:
    - (main_resistance, main_resistance_score): The main resistance price and its score.
    """
    # Get the main resistance row
    main_resistance_row = volume_profile_df[volume_profile_df['Label'] == 'Main Resistance']

    if not main_resistance_row.empty:
        main_resistance = main_resistance_row.iloc[0]['Price']
        main_resistance_score = main_resistance_row.iloc[0]['Score']
    else:
        # Fallback: find the closest non-zero score resistance above the current price
        fallback_resistance_row = volume_profile_df[
            (volume_profile_df['Price'] > current_price) & (volume_profile_df['Score'] > 0)
        ].sort_values(by='Price').head(1)

        # If no resistance with a non-zero score, choose the next available resistance line
        if fallback_resistance_row.empty:
            fallback_resistance_row = volume_profile_df[volume_profile_df['Price'] > current_price].sort_values(by='Price').head(1)

        if not fallback_resistance_row.empty:
            main_resistance = fallback_resistance_row.iloc[0]['Price']
            main_resistance_score = fallback_resistance_row.iloc[0]['Score']
        else:
            main_resistance = None
            main_resistance_score = None

    return main_resistance, main_resistance_score


# Helper function to compute the secondary resistance
def compute_secondary_resistance(volume_profile_df, main_resistance):
    """
    Computes the secondary resistance, which is the next resistance level after the main resistance.

    Args:
    - volume_profile_df (pd.DataFrame): DataFrame containing support and resistance levels.
    - main_resistance (float): The price of the main resistance.

    Returns:
    - (secondary_resistance, secondary_resistance_score): The secondary resistance price and its score.
    """
    # Find the next resistance level after the main resistance
    secondary_resistance_row = volume_profile_df[
        (volume_profile_df['Price'] > main_resistance) & (volume_profile_df['Label'] != 'Main Resistance')
    ].sort_values(by='Price').head(1)

    if not secondary_resistance_row.empty:
        secondary_resistance = secondary_resistance_row.iloc[0]['Price']
        secondary_resistance_score = secondary_resistance_row.iloc[0]['Score']
    else:
        secondary_resistance = None
        secondary_resistance_score = None

    return secondary_resistance, secondary_resistance_score

def calculate_stop_loss(current_price, atr_value, sma_50, main_support):
    """
    Calculate the stop loss as 1 ATR below the 50-day moving average or 1 ATR below the closest support,
    whichever is larger.

    Args:
    - current_price (float): The current price of the stock.
    - atr_value (float): The Average True Range (ATR) value.
    - sma_50 (float): The 50-day Simple Moving Average (SMA).
    - main_support (float): The closest support level.

    Returns:
    - stop_loss (float): The calculated stop loss.
    """
    # Calculate the two potential stop loss levels
    stop_loss_50_sma = sma_50 - atr_value if sma_50 is not None else None
    stop_loss_main_support = main_support - atr_value if main_support is not None else None
    
    # Use the greater of the two stop loss levels
    if stop_loss_50_sma is not None and stop_loss_main_support is not None:
        return max(stop_loss_50_sma, stop_loss_main_support)
    elif stop_loss_50_sma is not None:
        return stop_loss_50_sma
    elif stop_loss_main_support is not None:
        return stop_loss_main_support
    else:
        return None  # In case both SMA 50 and support are unavailable

def calculate_stop_loss(current_price, atr_value, sma_50, main_support):
    """
    Calculate the stop loss as 1 ATR below the 50-day moving average or 1 ATR below the closest support,
    whichever is larger. Ensure that the stop loss is below the current price.

    Args:
    - current_price (float): The current price of the stock.
    - atr_value (float): The Average True Range (ATR) value.
    - sma_50 (float): The 50-day Simple Moving Average (SMA).
    - main_support (float): The closest support level.

    Returns:
    - stop_loss (float): The calculated stop loss, or None if no valid stop loss is below the current price.
    """
    # Calculate the two potential stop loss levels
    stop_loss_50_sma = sma_50 - atr_value if sma_50 is not None else None
    stop_loss_main_support = main_support - atr_value if main_support is not None else None

    # Filter out stop losses that are above the current price
    stop_loss_candidates = [
        sl for sl in [stop_loss_50_sma, stop_loss_main_support] if sl is not None and sl < current_price
    ]

    # If there are valid stop loss candidates, return the maximum (as it should be as close to current price as possible)
    if stop_loss_candidates:
        return max(stop_loss_candidates)

    # If no valid stop loss is below the current price, return None
    return None

def calculate_number_of_shares(current_price, stop_loss, portfolio_size, risk_size, atr_value):
    """
    Calculate the number of shares to buy for a stock, based on the risk and constraints.

    Args:
    - current_price (float): The current price of the stock.
    - stop_loss (float): The calculated stop loss price.
    - portfolio_size (float): The total portfolio size.
    - risk_size (float): The percentage of the portfolio to risk on any trade.
    - atr_value (float): The Average True Range (ATR) value.

    Returns:
    - number_of_shares (float): The calculated number of shares to buy.
    """
    if stop_loss is None or stop_loss >= current_price:
        return 0  # If stop loss is invalid or higher than the current price, return 0 shares

    # Risk per share is the difference between current price and stop loss
    risk_per_share = current_price - stop_loss
    
    # Risk per trade (1% of portfolio size)
    risk_per_trade = portfolio_size * risk_size  # 1% of portfolio size is risked per trade

    # Calculate the number of shares based on risk
    number_of_shares_by_risk = risk_per_trade / risk_per_share if risk_per_share > 0 else 0

    # Maximum number of shares allowed based on 20% of the portfolio value
    max_shares_by_value = (0.20 * portfolio_size) / current_price

    # Final number of shares is the minimum of the risk-based number and the 20% rule
    return min(number_of_shares_by_risk, max_shares_by_value)

def calculate_technical_indicators(stock_data, volume_profile_df, selected_date, portfolio_size, risk_size=0.01, indicators_to_calculate=None):
    """
    Calculate technical indicators for the selected date using modular functions.
    Args:
    - stock_data (pd.DataFrame): Stock data containing 'Close', 'High', 'Low', 'ATR', etc.
    - volume_profile_df (pd.DataFrame): DataFrame containing support/resistance levels and Fibonacci levels.
    - selected_date (str): The date for which the technical indicators should be calculated.
    - portfolio_size (float): The total portfolio size.
    - risk_size (float): The percentage of the portfolio to risk on each trade (default is 1%).
    - indicators_to_calculate (list): List of indicator names to calculate (optional). Defaults to all.

    Returns:
    - A dictionary containing the calculated indicators, number of shares, and other key metrics.
    """
    selected_date = pd.to_datetime(selected_date)

    # Compute all technical indicators at once
    indicators = calculate_all_indicators(stock_data, selected_date)

    # Get the current price and relevant indicators
    current_price = stock_data.loc[selected_date]['Close']
    atr_value = indicators['ATR_14']

    # Calculate the 2-day trailing average of the 14-day ATR
    stock_data['ATR_14'] = ta.atr(stock_data['High'], stock_data['Low'], stock_data['Close'], length=14)
    stock_data['ATR_14_2D_AVG'] = stock_data['ATR_14'].rolling(window=2).mean()
    trailing_atr = stock_data.loc[selected_date, 'ATR_14_2D_AVG']

    # Calculate distances from moving averages if available
    sma_200 = indicators.get('SMA_200')
    sma_50 = indicators.get('SMA_50')
    sma_20 = indicators.get('SMA_20')
    
    distance_from_200_sma = (current_price - sma_200) / sma_200 * 100 if sma_200 and sma_200 > 0 else 0
    distance_from_50_sma = (current_price - sma_50) / sma_50 * 100 if sma_50 and sma_50 > 0 else 0
    distance_from_20_sma = (current_price - sma_20) / sma_20 * 100 if sma_20 and sma_20 > 0 else 0

    # Add RSI calculation
    stock_data['RSI'] = ta.rsi(stock_data['Close'], length=14)  # 14-day RSI
    rsi_value = stock_data.loc[selected_date]['RSI']  # Get RSI for the selected date

    # Compute main support using the helper function
    main_support, main_support_score, _ = compute_main_support(volume_profile_df, current_price, atr_value)

    # Compute main resistance using the helper function
    main_resistance, main_resistance_score = compute_main_resistance(volume_profile_df, current_price)

    # Compute secondary resistance using the helper function
    secondary_resistance, secondary_resistance_score = compute_secondary_resistance(volume_profile_df, main_resistance)

    # Calculate stop loss: 1 ATR below the 50-day moving average or the closest support
    stop_loss = calculate_stop_loss(current_price, atr_value, sma_50, main_support)

    # Calculate the number of shares based on risk
    number_of_shares = calculate_number_of_shares(current_price, stop_loss, portfolio_size, risk_size, trailing_atr)

    # Calculate the risk-reward ratio and suggested limit price for the main resistance
    risk_reward_ratio, estimated_gain, suggested_limit_price = calculate_risk_reward(current_price, stop_loss, main_resistance)

    # Calculate the risk-reward ratio and suggested limit price for the secondary resistance
    secondary_risk_reward_ratio, secondary_estimated_gain, secondary_suggested_limit_price = calculate_risk_reward(
        current_price, stop_loss, secondary_resistance
    )

    # Calculate the secondary estimated gain (percentage increase from current price to secondary resistance)
    if secondary_resistance is not None and current_price > 0:
        secondary_estimated_gain = (secondary_resistance - current_price) / current_price * 100
    else:
        secondary_estimated_gain = None

    # Collect candlestick patterns with values that are not zero
    candlestick_patterns = indicators.get('Candlestick_Patterns', {})
    patterns = candlestick_patterns[candlestick_patterns != 0].to_dict() if isinstance(candlestick_patterns, pd.Series) else {}

    # Format the candlestick patterns for better readability (comma-separated and sorted by value)
    formatted_patterns = format_candlestick_patterns(patterns)

    return {
        'Current Price': current_price,
        'RSI': rsi_value,  # Add RSI to the output
        'Main Support': main_support,
        'Main Support Score': main_support_score,  # Add main support score
        'Main Resistance': main_resistance,
        'Main Resistance Score': main_resistance_score,  # Add main resistance score
        'Stop Loss': stop_loss,  # Add the calculated stop loss
        'Number of Shares': number_of_shares,  # Add the calculated number of shares
        'Risk-Reward Ratio': risk_reward_ratio,
        'Estimated Gain (%)': estimated_gain,  # Estimated gain for main resistance
        'Suggested Limit Price': suggested_limit_price,
        'Secondary Resistance': secondary_resistance,  # Add secondary resistance
        'Secondary Resistance Score': secondary_resistance_score,  # Add secondary resistance score
        'Secondary Risk-Reward Ratio': secondary_risk_reward_ratio,  # Add secondary risk-reward ratio
        'Secondary Estimated Gain (%)': secondary_estimated_gain,  # Add secondary estimated gain
        'Secondary Suggested Limit Price': secondary_suggested_limit_price,  # Add secondary suggested limit price
        'MACD Histogram': indicators.get('MACD_Histogram'),
        'Distance from 200-day SMA (%)': distance_from_200_sma,
        'Distance from 50-day SMA (%)': distance_from_50_sma,
        'Distance from 20-day SMA (%)': distance_from_20_sma,
        'ATR': atr_value,
        'Candlestick Patterns': formatted_patterns  # Add the formatted candlestick patterns to the output
    }

In [69]:
def compute_volume_profile_with_bins(stock_data, end_date, bins=50):
    """
    Compute the volume profile for both six months and three months, smooth them,
    and compute the volume velocity for each peak.

    Args:
    - stock_data (pd.DataFrame): Stock data containing 'Close', 'High', 'Low', and 'Volume'.
    - end_date (str): End date for the six-month period (format: 'YYYY-MM-DD').
    - bins (int): Number of bins for the volume profile (default is 50).

    Returns:
    - volume_profile_df (pd.DataFrame): Dataframe containing major and minor support/resistance with both six-month and three-month scores and volume velocity.
    - smoothed_volume_profile_six (np.array): Smoothed six-month volume profile.
    - smoothed_volume_profile_three (np.array): Smoothed three-month volume profile.
    - volume_profile_six (np.array): Raw six-month volume profile.
    - volume_profile_three (np.array): Raw three-month volume profile.
    - price_bins (np.array): Price bins used to compute the volume profile.
    """
    # Compute six-month volume profile and price bins
    volume_profile_six, smoothed_volume_profile_six, price_bins = compute_and_smooth_volume_profile(stock_data, end_date, bins, months=6)

    # Ensure three-month volume profile uses the same price_bins
    volume_profile_three, smoothed_volume_profile_three = compute_volume_profile_with_existing_bins(stock_data, end_date, price_bins, months=3)

    # Find local maxima in the six-month profile
    volume_profile_df = find_local_maxima(price_bins, smoothed_volume_profile_six)

    # Compute the three-month score using the three-month volume profile
    volume_profile_df['Three-Month Score'] = compute_three_month_scores(smoothed_volume_profile_three, price_bins, volume_profile_df['Price'])

    # Replace any NaN values in the 'Three-Month Score' column with 0
    volume_profile_df['Three-Month Score'] = volume_profile_df['Three-Month Score'].replace(np.nan, 0)

    # Compute the volume velocity for each peak based on the difference between the six-month and three-month scores
    volume_profile_df = compute_volume_velocity_from_scores(volume_profile_df)

    # Identify main support and resistance
    current_price = stock_data['Close'].iloc[-1]
    volume_profile_df = identify_support_resistance(volume_profile_df, current_price)

    return volume_profile_df, smoothed_volume_profile_six, smoothed_volume_profile_three, volume_profile_six, volume_profile_three, price_bins    

def filter_stock_data(stock_data, end_date):
    """
    Filters the stock data to include only the six months before the given end date.

    Args:
    - stock_data (pd.DataFrame): The full stock data.
    - end_date (str): The end date for filtering (format: 'YYYY-MM-DD').

    Returns:
    - pd.DataFrame: Filtered stock data for the last six months.
    """
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(months=6)
    return stock_data[(stock_data.index >= start_date) & (stock_data.index <= end_date)]


def compute_price_bins_and_volume_profile(stock_data, bins):
    """
    Computes the price bins and the volume profile.

    Args:
    - stock_data (pd.DataFrame): The filtered stock data.
    - bins (int): Number of bins for the volume profile.

    Returns:
    - np.array: Price bins.
    - np.array: Volume profile for each price bin.
    """
    high_prices = stock_data['High']
    low_prices = stock_data['Low']
    close_prices = stock_data['Close']
    volume = stock_data['Volume']

    # Compute price bins
    price_bins = np.linspace(low_prices.min(), high_prices.max(), bins)

    # Compute volume profile for each bin
    volume_profile = []
    for i in range(len(price_bins) - 1):
        bin_mask = (close_prices > price_bins[i]) & (close_prices <= price_bins[i + 1])
        volume_profile.append(volume[bin_mask].sum())
    
    return price_bins, np.array(volume_profile)


def smooth_volume_profile(volume_profile, sigma=2):
    """
    Smooths the volume profile using a Gaussian filter.

    Args:
    - volume_profile (np.array): The raw volume profile.
    - sigma (int): The smoothing parameter for the Gaussian filter (default is 2).

    Returns:
    - np.array: The smoothed volume profile.
    """
    return gaussian_filter(volume_profile, sigma=sigma)


def find_local_maxima(price_bins, smoothed_volume_profile):
    """
    Finds local maxima in the smoothed volume profile and computes the prominence scores.

    Args:
    - price_bins (np.array): Price bins used for volume profile.
    - smoothed_volume_profile (np.array): The smoothed volume profile.

    Returns:
    - pd.DataFrame: DataFrame containing the local maxima and their normalized scores.
    """
    # Find local maxima
    local_maxima_indices = argrelextrema(smoothed_volume_profile, np.greater)[0]
    local_maxima_prices = price_bins[local_maxima_indices]
    
    # Calculate prominence/score for each local maxima
    prominence_scores = smoothed_volume_profile[local_maxima_indices]
    max_prominence = np.max(prominence_scores) if len(prominence_scores) > 0 else 1
    normalized_scores = prominence_scores / max_prominence
    
    # Create a dataframe of local maxima
    return pd.DataFrame({
        'Price': local_maxima_prices,
        'Score': normalized_scores
    })


def identify_support_resistance(volume_profile_df, current_price):
    """
    Identifies main and minor support/resistance levels based on the current price.

    Args:
    - volume_profile_df (pd.DataFrame): DataFrame containing local maxima and their scores.
    - current_price (float): The current price of the stock.

    Returns:
    - pd.DataFrame: Updated DataFrame with support/resistance labels.
    """
    # Main support: Closest local maxima below the current price with score > 0.4
    support_candidates = volume_profile_df[volume_profile_df['Price'] < current_price]
    main_support = support_candidates[support_candidates['Score'] > 0.4].sort_values(by='Price', ascending=False).head(1)
    
    if main_support.empty:
        main_support = support_candidates.sort_values(by='Score', ascending=False).head(1) if not support_candidates.empty else pd.DataFrame()

    # Main resistance: Closest local maxima above the current price with score > 0.4
    resistance_candidates = volume_profile_df[volume_profile_df['Price'] > current_price]
    main_resistance = resistance_candidates[resistance_candidates['Score'] > 0.4].sort_values(by='Price').head(1)
    
    if main_resistance.empty:
        main_resistance = resistance_candidates.sort_values(by='Score', ascending=False).head(1) if not resistance_candidates.empty else pd.DataFrame()

    # Label main support and resistance
    if not main_support.empty:
        volume_profile_df.loc[main_support.index, 'Label'] = 'Main Support'
    
    if not main_resistance.empty:
        volume_profile_df.loc[main_resistance.index, 'Label'] = 'Main Resistance'

    # Label minor supports and resistances
    if not main_support.empty:
        minor_supports = support_candidates[support_candidates['Price'] != main_support['Price'].values[0]].sort_values(by='Price', ascending=False)
        volume_profile_df.loc[minor_supports.index, 'Label'] = 'Minor Support'
    
    if not main_resistance.empty:
        minor_resistances = resistance_candidates[resistance_candidates['Price'] != main_resistance['Price'].values[0]].sort_values(by='Price')
        volume_profile_df.loc[minor_resistances.index, 'Label'] = 'Minor Resistance'
    
    return volume_profile_df

def plot_volume_profile_with_support_resistance(stock_data, volume_profile_df, smoothed_volume_profile_six, smoothed_volume_profile_three, price_bins, end_date):
    """
    Plots the original and smoothed volume profiles along with the main support and resistance lines.
    """
    # Filter the stock data for the last six months
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(months=6)
    stock_data = stock_data[(stock_data.index >= start_date) & (stock_data.index <= end_date)]

    # Extract relevant data
    close_prices = stock_data['Close']

    plt.figure(figsize=(12, 8))

    # Plot original and smoothed volume profiles for six months
    plt.subplot(2, 1, 1)
    plt.barh(price_bins[:-1], smoothed_volume_profile_six, height=(price_bins[1] - price_bins[0]), color='blue', alpha=0.6, label='6-Month Smoothed Volume')
    plt.plot(smoothed_volume_profile_six, price_bins[:-1], color='red', linewidth=2, label='6-Month Smoothed Volume')

    # Plot original and smoothed volume profiles for three months
    plt.subplot(2, 1, 2)
    plt.barh(price_bins[:-1], smoothed_volume_profile_three, height=(price_bins[1] - price_bins[0]), color='green', alpha=0.6, label='3-Month Smoothed Volume')
    plt.plot(smoothed_volume_profile_three, price_bins[:-1], color='purple', linewidth=2, label='3-Month Smoothed Volume')

    # Add support and resistance lines and volume velocity annotations
    for _, row in volume_profile_df.iterrows():
        plt.axhline(y=row['Price'], color='gray', linestyle='--', linewidth=1)
        plt.text(smoothed_volume_profile_six[0], row['Price'], f"Velocity: {row['Volume Velocity']:.2f}", color='black')

    # Plot the current price as a black horizontal line
    current_price = close_prices.iloc[-1]
    plt.axhline(y=current_price, color='black', linestyle='-', linewidth=2, label='Current Price')

    plt.xlabel('Volume')
    plt.ylabel('Price')
    plt.legend()
    plt.tight_layout()
    plt.show()

def compute_volume_velocity_from_scores(volume_profile_df):
    """
    Computes the volume velocity for each peak by comparing the three-month score to the six-month score.

    Args:
    - volume_profile_df (pd.DataFrame): DataFrame containing the peaks from the six-month profile, including both six-month and three-month scores.

    Returns:
    - volume_profile_df (pd.DataFrame): Updated DataFrame with volume velocity added.
    """
    velocities = []

    for _, row in volume_profile_df.iterrows():
        # Compute velocity as the difference between the three-month score and the six-month score
        velocity = row['Three-Month Score'] - row['Score']
        velocities.append(velocity)

    # Add velocity to the DataFrame
    volume_profile_df['Volume Velocity'] = velocities

    return volume_profile_df
    
def compute_volume_profile_with_existing_bins(stock_data, end_date, price_bins, months):
    """
    Compute the volume profile using existing price bins for the specified time period.

    Args:
    - stock_data (pd.DataFrame): The filtered stock data.
    - end_date (str): The end date for filtering (format: 'YYYY-MM-DD').
    - price_bins (np.array): The price bins for the volume profile (shared between six-month and three-month profiles).
    - months (int): The number of months to consider for the volume profile.

    Returns:
    - volume_profile (np.array): The raw volume profile.
    - smoothed_volume_profile (np.array): The smoothed volume profile.
    """
    # Filter the stock data for the specified months prior to the end_date
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(months=months)
    filtered_data = stock_data[(stock_data.index >= start_date) & (stock_data.index <= end_date)]

    # Use the provided price_bins to compute the volume profile
    volume_profile = []
    for i in range(len(price_bins) - 1):
        bin_mask = (filtered_data['Close'] > price_bins[i]) & (filtered_data['Close'] <= price_bins[i + 1])
        volume_profile.append(filtered_data['Volume'][bin_mask].sum())

    volume_profile = np.array(volume_profile)

    # Smooth the volume profile
    smoothed_volume_profile = gaussian_filter(volume_profile, sigma=2)

    return volume_profile, smoothed_volume_profile

def compute_and_smooth_volume_profile(stock_data, end_date, bins, months):
    """
    Computes the volume profile and smooths it for the specified time period (in months).

    Args:
    - stock_data (pd.DataFrame): The filtered stock data.
    - end_date (str): The end date for filtering (format: 'YYYY-MM-DD').
    - bins (int): Number of bins for the volume profile.
    - months (int): The number of months to consider for the volume profile.

    Returns:
    - volume_profile (np.array): The raw volume profile.
    - smoothed_volume_profile (np.array): The smoothed volume profile.
    - price_bins (np.array): The price bins for the volume profile.
    """
    # Filter the stock data for the specified months prior to the end_date
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(months=months)
    filtered_data = stock_data[(stock_data.index >= start_date) & (stock_data.index <= end_date)]

    # Compute price bins and volume profile
    price_bins, volume_profile = compute_price_bins_and_volume_profile(filtered_data, bins)

    # Smooth the volume profile
    smoothed_volume_profile = smooth_volume_profile(volume_profile, sigma=2)

    return volume_profile, smoothed_volume_profile, price_bins

def compute_volume_velocity(volume_profile_df, price_bins, smoothed_volume_profile_six, smoothed_volume_profile_three):
    """
    Computes the volume velocity for each peak by comparing the proportion of volume in the six-month
    and three-month smoothed volume profiles.

    Args:
    - volume_profile_df (pd.DataFrame): DataFrame containing the peaks from the six-month profile.
    - price_bins (np.array): The price bins for the volume profile.
    - smoothed_volume_profile_six (np.array): The smoothed six-month volume profile.
    - smoothed_volume_profile_three (np.array): The smoothed three-month volume profile.

    Returns:
    - volume_profile_df (pd.DataFrame): Updated DataFrame with volume velocity added.
    """
    max_six = np.max(smoothed_volume_profile_six)
    max_three = np.max(smoothed_volume_profile_three)

    velocities = []

    for _, row in volume_profile_df.iterrows():
        price_level = row['Price']
        closest_bin_idx = np.abs(price_bins - price_level).argmin()

        # Get proportional volume for six-month and three-month
        volume_six = smoothed_volume_profile_six[closest_bin_idx] / max_six if max_six > 0 else 0
        volume_three = smoothed_volume_profile_three[closest_bin_idx] / max_three if max_three > 0 else 0

        # Compute velocity (positive if three-month volume is higher, negative if lower)
        velocity = volume_three - volume_six
        velocities.append(velocity)

    # Add velocity to the DataFrame
    volume_profile_df['Volume Velocity'] = velocities

    return volume_profile_df
    
def compute_three_month_scores(volume_profile_three, price_bins, price_levels):
    """
    Compute the three-month score for each peak based on the three-month volume profile.

    Args:
    - volume_profile_three (np.array): The smoothed three-month volume profile.
    - price_bins (np.array): The price bins for the volume profile.
    - price_levels (np.array): The price levels of the local maxima from the six-month profile.

    Returns:
    - np.array: The three-month scores corresponding to each price level.
    """
    three_month_scores = []
    max_three = np.max(volume_profile_three)

    for price_level in price_levels:
        closest_bin_idx = np.abs(price_bins - price_level).argmin()
        volume_three = volume_profile_three[closest_bin_idx]

        # Normalize the three-month score
        score = volume_three / max_three if max_three > 0 else 0
        three_month_scores.append(score)

    return three_month_scores
    
def filter_stock_data(stock_data, end_date):
    """
    Filters the stock data to include only the six months before the given end date.

    Args:
    - stock_data (pd.DataFrame): The full stock data.
    - end_date (str): The end date for filtering (format: 'YYYY-MM-DD').

    Returns:
    - pd.DataFrame: Filtered stock data for the last six months.
    """
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(months=6)
    return stock_data[(stock_data.index >= start_date) & (stock_data.index <= end_date)]


def compute_price_bins_and_volume_profile(stock_data, bins):
    """
    Computes the price bins and the volume profile.

    Args:
    - stock_data (pd.DataFrame): The filtered stock data.
    - bins (int): Number of bins for the volume profile.

    Returns:
    - np.array: Price bins.
    - np.array: Volume profile for each price bin.
    """
    high_prices = stock_data['High']
    low_prices = stock_data['Low']
    close_prices = stock_data['Close']
    volume = stock_data['Volume']

    # Compute price bins
    price_bins = np.linspace(low_prices.min(), high_prices.max(), bins)

    # Compute volume profile for each bin
    volume_profile = []
    for i in range(len(price_bins) - 1):
        bin_mask = (close_prices > price_bins[i]) & (close_prices <= price_bins[i + 1])
        volume_profile.append(volume[bin_mask].sum())
    
    return price_bins, np.array(volume_profile)


def smooth_volume_profile(volume_profile, sigma=2):
    """
    Smooths the volume profile using a Gaussian filter.

    Args:
    - volume_profile (np.array): The raw volume profile.
    - sigma (int): The smoothing parameter for the Gaussian filter (default is 2).

    Returns:
    - np.array: The smoothed volume profile.
    """
    return gaussian_filter(volume_profile, sigma=sigma)


def find_local_maxima(price_bins, smoothed_volume_profile):
    """
    Finds local maxima in the smoothed volume profile and computes the prominence scores.

    Args:
    - price_bins (np.array): Price bins used for volume profile.
    - smoothed_volume_profile (np.array): The smoothed volume profile.

    Returns:
    - pd.DataFrame: DataFrame containing the local maxima and their normalized scores.
    """
    # Find local maxima
    local_maxima_indices = argrelextrema(smoothed_volume_profile, np.greater)[0]
    local_maxima_prices = price_bins[local_maxima_indices]
    
    # Calculate prominence/score for each local maxima
    prominence_scores = smoothed_volume_profile[local_maxima_indices]
    max_prominence = np.max(prominence_scores) if len(prominence_scores) > 0 else 1
    normalized_scores = prominence_scores / max_prominence
    
    # Create a dataframe of local maxima
    return pd.DataFrame({
        'Price': local_maxima_prices,
        'Score': normalized_scores
    })


def identify_support_resistance(volume_profile_df, current_price):
    """
    Identifies main and minor support/resistance levels based on the current price.

    Args:
    - volume_profile_df (pd.DataFrame): DataFrame containing local maxima and their scores.
    - current_price (float): The current price of the stock.

    Returns:
    - pd.DataFrame: Updated DataFrame with support/resistance labels.
    """
    # Main support: Closest local maxima below the current price with score > 0.4
    support_candidates = volume_profile_df[volume_profile_df['Price'] < current_price]
    main_support = support_candidates[support_candidates['Score'] > 0.4].sort_values(by='Price', ascending=False).head(1)
    
    if main_support.empty:
        main_support = support_candidates.sort_values(by='Score', ascending=False).head(1) if not support_candidates.empty else pd.DataFrame()

    # Main resistance: Closest local maxima above the current price with score > 0.4
    resistance_candidates = volume_profile_df[volume_profile_df['Price'] > current_price]
    main_resistance = resistance_candidates[resistance_candidates['Score'] > 0.4].sort_values(by='Price').head(1)
    
    if main_resistance.empty:
        main_resistance = resistance_candidates.sort_values(by='Score', ascending=False).head(1) if not resistance_candidates.empty else pd.DataFrame()

    # Label main support and resistance
    if not main_support.empty:
        volume_profile_df.loc[main_support.index, 'Label'] = 'Main Support'
    
    if not main_resistance.empty:
        volume_profile_df.loc[main_resistance.index, 'Label'] = 'Main Resistance'

    # Label minor supports and resistances
    if not main_support.empty:
        minor_supports = support_candidates[support_candidates['Price'] != main_support['Price'].values[0]].sort_values(by='Price', ascending=False)
        volume_profile_df.loc[minor_supports.index, 'Label'] = 'Minor Support'
    
    if not main_resistance.empty:
        minor_resistances = resistance_candidates[resistance_candidates['Price'] != main_resistance['Price'].values[0]].sort_values(by='Price')
        volume_profile_df.loc[minor_resistances.index, 'Label'] = 'Minor Resistance'
    
    return volume_profile_df

In [73]:

def plot_full_price_chart_with_fibonacci(stock_data, selected_date, volume_profile_df):
    """
    Plots the full price history, adding:
    - A black vertical line at the selected date.
    - Main and minor support and resistance levels with annotations showing their scores.
    - Fibonacci levels 0, 1, 1.236, and 1.382 with their corresponding score.
    """
    plt.figure(figsize=(14, 6))
    
    # Plot the full price history
    plt.plot(stock_data.index, stock_data['Close'], label='Close Price', color='blue')
    
    # Draw a black vertical line at the selected date
    plt.axvline(pd.to_datetime(selected_date), color='black', linestyle='-', linewidth=2, label=f'Selected Date: {selected_date}')
    
    # Leftmost date to position the annotations
    leftmost_date = stock_data.index[0]
    
    # Plot support, resistance, and Fibonacci levels from volume_profile_df
    for _, row in volume_profile_df.iterrows():
        if row['Is Fibonacci']:
            # Plot Fibonacci levels with score annotation
            plt.axhline(y=row['Price'], color='purple', linestyle='--', linewidth=1.5, label=f'Fib {row["Label"]}: {row["Price"]:.2f}')
            plt.text(leftmost_date, row['Price'], f"Fib {row['Label']} (Score: {row['Score']:.2f})", color='purple', verticalalignment='center', horizontalalignment='right', fontsize=8)
        else:
            # Plot main and minor support/resistance levels
            color = 'green' if 'Support' in row['Label'] else 'red'
            linestyle = '--' if 'Main' in row['Label'] else ':'
            plt.axhline(y=row['Price'], color=color, linestyle=linestyle, linewidth=2 if 'Main' in row['Label'] else 1)
            plt.text(leftmost_date, row['Price'], f"Score: {row['Score']:.2f}", color=color, verticalalignment='center', horizontalalignment='right')
    
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title('Full Price History with Support/Resistance and Fibonacci Levels')
    plt.legend()
    plt.tight_layout()
    plt.show()
        
def append_fibonacci_levels_to_df(stock_data, volume_profile_df, smoothed_volume_profile, price_bins, end_date):
    """
    Computes the Fibonacci retracement levels (0%, 100%) and extension levels (1.236, 1.382).
    Adds these levels to the volume profile DataFrame and computes the score as the volume divided by the maximum smoothed volume.
    Merges levels that are within 1% of each other.
    
    Args:
    - stock_data (pd.DataFrame): Stock data containing 'High' and 'Low' prices.
    - volume_profile_df (pd.DataFrame): DataFrame containing existing support and resistance levels.
    - smoothed_volume_profile (np.array): Smoothed volume profile to get volume for Fibonacci levels.
    - price_bins (np.array): Price bins used to compute the volume profile.
    - end_date (str): The end date for calculating Fibonacci levels.
    
    Returns:
    - volume_profile_df (pd.DataFrame): Updated DataFrame with Fibonacci levels appended and labeled, merged within 1%.
    """
    # Filter the stock data to only include the last 6 months
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(months=6)
    stock_data = stock_data[(stock_data.index >= start_date) & (stock_data.index <= end_date)]
    
    # Find the highest and lowest price over the last 6 months
    highest_price = stock_data['High'].max()
    lowest_price = stock_data['Low'].min()

    # Set Fibonacci levels as 0%, 100%, and the extensions 1.236 and 1.382
    fib_levels_percent = [0, 1, 1.236, 1.382]
    
    # Calculate the price levels corresponding to the Fibonacci levels
    fib_levels = []
    max_smoothed_volume = np.max(smoothed_volume_profile)  # Find the maximum smoothed volume

    for percent in fib_levels_percent:
        level = lowest_price + (highest_price - lowest_price) * percent
        
        # Check if the Fibonacci level is outside the price bins
        if level > price_bins[-1] or level < price_bins[0]:
            # If outside the range, no corresponding volume data (assign score 0)
            corresponding_volume = 0
            score = 0
        else:
            # Find the closest price bin to the Fibonacci level to get the corresponding volume
            closest_bin_idx = np.abs(price_bins - level).argmin()

            # Adjust for the Fibonacci 1 level (100%) by subtracting one from the index
            if percent == 1 and closest_bin_idx == len(price_bins) - 1:
                closest_bin_idx -= 1

            corresponding_volume = smoothed_volume_profile[closest_bin_idx]
            
            # Calculate the score as the volume divided by the maximum smoothed volume
            score = corresponding_volume / max_smoothed_volume if max_smoothed_volume > 0 else 0

        fib_levels.append({
            'Price': level,
            'Score': score,  # Use score instead of raw volume
            'Label': f"Fib {percent:.3f}",
            'Is Fibonacci': True,  # Mark as a Fibonacci level
            'Volume': corresponding_volume  # Store the volume for reference
        })
    
    # Create a DataFrame for Fibonacci levels
    fib_levels_df = pd.DataFrame(fib_levels)
    
    # Ensure the volume_profile_df has the 'Is Fibonacci' column and set to False for existing levels
    if 'Is Fibonacci' not in volume_profile_df.columns:
        volume_profile_df['Is Fibonacci'] = False  # Mark existing levels as not Fibonacci
    
    # Concatenate the volume profile DataFrame with the Fibonacci levels DataFrame
    volume_profile_df = pd.concat([volume_profile_df, fib_levels_df], ignore_index=True, sort=False)

    # Sort the DataFrame by 'Price' for easy comparison of consecutive levels
    volume_profile_df = volume_profile_df.sort_values(by='Price').reset_index(drop=True)

    # Merging lines within 1% of each other
    merged_rows = []
    skip_next = False

    for i in range(len(volume_profile_df) - 1):
        if skip_next:
            skip_next = False
            continue

        current_row = volume_profile_df.iloc[i]
        next_row = volume_profile_df.iloc[i + 1]

        # Calculate percentage difference between consecutive prices
        price_diff = abs(next_row['Price'] - current_row['Price']) / current_row['Price']

        if price_diff <= 0.02:  # If within 1% range
            # Merge: Average the prices, and you can either sum or average the scores
            merged_price = (current_row['Price'] + next_row['Price']) / 2
            merged_score = (current_row['Score'] + next_row['Score']) / 2  # Average score
            merged_label = current_row['Label'] if 'Main' in current_row['Label'] else next_row['Label']

            merged_rows.append({
                'Price': merged_price,
                'Score': merged_score,
                'Label': merged_label,
                'Is Fibonacci': current_row['Is Fibonacci'] or next_row['Is Fibonacci'],
                'Volume': current_row['Volume'] + next_row['Volume']  # Sum the volumes
            })

            skip_next = True  # Skip the next row as it's already merged
        else:
            merged_rows.append(current_row.to_dict())

    # Add the last row if it wasn't merged
    if not skip_next:
        merged_rows.append(volume_profile_df.iloc[-1].to_dict())

    # Convert merged rows back to DataFrame
    volume_profile_df = pd.DataFrame(merged_rows)

    return volume_profile_df

def plot_volume_profile_with_support_resistance(stock_data, volume_profile_df, smoothed_volume_profile_six, smoothed_volume_profile_three, raw_volume_profile_six, raw_volume_profile_three, price_bins, end_date):
    """
    Plots the original (raw) and smoothed volume profiles along with the main support, resistance,
    and secondary support/resistance lines. Annotates the peaks from the six-month chart
    on both the three-month and six-month charts.

    Args:
    - stock_data (pd.DataFrame): Stock data for the stock.
    - volume_profile_df (pd.DataFrame): DataFrame containing volume profile peaks and their information.
    - smoothed_volume_profile_six (np.array): Smoothed six-month volume profile.
    - smoothed_volume_profile_three (np.array): Smoothed three-month volume profile.
    - raw_volume_profile_six (np.array): Raw six-month volume profile.
    - raw_volume_profile_three (np.array): Raw three-month volume profile.
    - price_bins (np.array): Price bins used to compute the volume profile.
    - end_date (pd.Timestamp): End date for the profile computation.
    """
    # Filter the stock data for the last six months and last three months
    end_date = pd.to_datetime(end_date)
    start_date_six_months = end_date - pd.DateOffset(months=6)
    start_date_three_months = end_date - pd.DateOffset(months=3)
    
    # Filter stock data for six and three months
    stock_data_six_months = stock_data[(stock_data.index >= start_date_six_months) & (stock_data.index <= end_date)]
    stock_data_three_months = stock_data[(stock_data.index >= start_date_three_months) & (stock_data.index <= end_date)]

    # Extract relevant data
    close_prices = stock_data_six_months['Close']

    # Create a 1x2 grid for six-month and three-month plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

    # Plot the 6-month volume profile (raw as bars, smoothed as envelope)
    ax1.barh(price_bins[:-1], raw_volume_profile_six, height=(price_bins[1] - price_bins[0]), color='blue', alpha=0.6, label='6-Month Raw Volume')
    ax1.plot(smoothed_volume_profile_six, price_bins[:-1], color='red', linewidth=2, label='6-Month Smoothed Volume')
    ax1.set_title(f"6-Month Volume Profile (Up to {end_date.date()})")
    ax1.set_xlabel('Volume')
    ax1.set_ylabel('Price')
    ax1.legend()

    # Plot the 3-month volume profile (raw as bars, smoothed as envelope)
    ax2.barh(price_bins[:-1], raw_volume_profile_three, height=(price_bins[1] - price_bins[0]), color='green', alpha=0.6, label='3-Month Raw Volume')
    ax2.plot(smoothed_volume_profile_three, price_bins[:-1], color='purple', linewidth=2, label='3-Month Smoothed Volume')
    ax2.set_title(f"3-Month Volume Profile (Up to {end_date.date()})")
    ax2.set_xlabel('Volume')
    ax2.legend()

    # Annotate peaks on both charts and draw lines at peak locations
    for _, row in volume_profile_df.iterrows():
        # Ensure the corresponding_bin_idx is within bounds
        corresponding_bin_idx_six = np.abs(price_bins - row['Price']).argmin()
        if corresponding_bin_idx_six >= len(smoothed_volume_profile_six):
            corresponding_bin_idx_six = len(smoothed_volume_profile_six) - 1
        
        corresponding_bin_idx_three = np.abs(price_bins - row['Price']).argmin()
        if corresponding_bin_idx_three >= len(smoothed_volume_profile_three):
            corresponding_bin_idx_three = len(smoothed_volume_profile_three) - 1

        # Exclude Fibonacci levels beyond 1.000
        if 'Fib' in row['Label'] and row['Price'] > price_bins[-1]:  # Skip lines beyond Fib 1.000
            continue

        # Annotate the score on the six-month chart (use six-month data)
        volume_value_six = smoothed_volume_profile_six[corresponding_bin_idx_six]
        ax1.text(volume_value_six * 0.7, row['Price'], f"Score: {row['Score']:.2f}",
                 color='black', verticalalignment='center', fontsize=8, bbox=dict(facecolor='white', edgecolor='none', pad=1))

        # Annotate the score and velocity on the three-month chart (use three-month data)
        volume_value_three = smoothed_volume_profile_three[corresponding_bin_idx_three]
        ax2.text(volume_value_three * 0.7, row['Price'], f"Score: {row['Three-Month Score']:.2f}, Vel: {row['Volume Velocity']:.2f}",
                 color='black', verticalalignment='center', fontsize=8, bbox=dict(facecolor='white', edgecolor='none', pad=1))

        # Draw lines at the peaks (only up to Fib 1.000)
        ax1.axhline(y=row['Price'], color='gray', linestyle='--', linewidth=1)
        ax2.axhline(y=row['Price'], color='gray', linestyle='--', linewidth=1)

    # Add support and resistance lines for both subplots (6-month on ax1 and 3-month on ax2)
    for _, row in volume_profile_df.iterrows():
        if 'Support' in row['Label']:
            color = 'green' if 'Main' in row['Label'] else 'lightgreen'
        elif 'Resistance' in row['Label']:
            color = 'red' if 'Main' in row['Label'] or 'Secondary' in row['Label'] else 'lightcoral'
        else:
            color = 'gray'

        # Draw lines on both the six-month (ax1) and three-month (ax2) plots
        ax1.axhline(y=row['Price'], color=color, linestyle='--', linewidth=2 if 'Main' in row['Label'] else 1)
        ax2.axhline(y=row['Price'], color=color, linestyle='--', linewidth=2 if 'Main' in row['Label'] else 1)

    # Plot the current price as a black horizontal line on both subplots
    current_price = close_prices.iloc[-1]
    ax1.axhline(y=current_price, color='black', linestyle='-', linewidth=2, label='Current Price')
    ax2.axhline(y=current_price, color='black', linestyle='-', linewidth=2, label='Current Price')

    plt.tight_layout()
    plt.show()
    
# Function to display technical indicators in the table
def display_technical_indicators_table(indicators):
    indicators_df = pd.DataFrame([indicators])
    display(indicators_df)


def plot_candlestick_chart(symbol, end_date, df):
    global stock_data  # Ensure we use the global stock_data
    end_date = pd.to_datetime(end_date)
    start_date = end_date - pd.DateOffset(months=6)  # Six months back from the selected date
    
    # Filter stock data for six months up to the selected end date
    six_months_data = stock_data.loc[(stock_data.index >= start_date) & (stock_data.index <= end_date)].copy()

    # Calculate moving averages using the full history, not just the six-month data
    stock_data['SMA_200'] = ta.sma(stock_data['Close'], length=200)
    stock_data['SMA_50'] = ta.sma(stock_data['Close'], length=50)
    stock_data['SMA_20'] = ta.sma(stock_data['Close'], length=20)

    # Merge the full history of moving averages into the six-month data
    six_months_data['SMA_200'] = stock_data['SMA_200']
    six_months_data['SMA_50'] = stock_data['SMA_50']
    six_months_data['SMA_20'] = stock_data['SMA_20']

    # Define moving average plots
    moving_averages = [
        mpf.make_addplot(six_months_data['SMA_200'], color='blue'),
        mpf.make_addplot(six_months_data['SMA_50'], color='green'),
        mpf.make_addplot(six_months_data['SMA_20'], color='orange')
    ]

    # Get all support, resistance, and Fibonacci levels from the DataFrame
    support_lines = df[df['Label'].str.contains('Support')]
    resistance_lines = df[df['Label'].str.contains('Resistance')]
    fibonacci_lines = df[df['Is Fibonacci'] == True]  # Extract Fibonacci levels

    # Define support and resistance lines (main support/resistance will appear thicker)
    support_resistance_lines = []
    
    # Plot support lines (Always green)
    for _, row in support_lines.iterrows():
        color = 'green' if 'Main' in row['Label'] else 'lightgreen'  # Support is always green
        support_resistance_lines.append(
            mpf.make_addplot([row['Price']] * len(six_months_data), color=color, linestyle='--', secondary_y=False)
        )
    
    # Plot resistance lines (Always red)
    for _, row in resistance_lines.iterrows():
        color = 'red' if 'Main' in row['Label'] else 'lightcoral'  # Resistance is always red
        support_resistance_lines.append(
            mpf.make_addplot([row['Price']] * len(six_months_data), color=color, linestyle='--', secondary_y=False)
        )
    
    # Plot Fibonacci lines (Purple for Fibonacci)
    for _, row in fibonacci_lines.iterrows():
        support_resistance_lines.append(
            mpf.make_addplot([row['Price']] * len(six_months_data), color='purple', linestyle=':', secondary_y=False)
        )
    
    # Combine moving averages, support/resistance, and Fibonacci lines
    addplots = moving_averages + support_resistance_lines
    
    # Calculate y-axis padding (e.g., 10% above the max price)
    min_price = six_months_data['Low'].min()
    max_price = six_months_data['High'].max()
    y_padding = (max_price - min_price) * 0.10  # 10% padding
    
    # Create the candlestick plot with moving averages and support/resistance lines
    fig, ax = mpf.plot(six_months_data, type='candle', style='charles', volume=True, 
             addplot=addplots, title=f"{symbol} (up to {end_date.date()})", 
             ylabel='Price', ylabel_lower='Volume', show_nontrading=True, 
             figratio=(16, 9), figscale=1.5, returnfig=True, 
             ylim=(min_price, max_price + y_padding))  # Add y-axis padding

    # Annotate the support, resistance, and Fibonacci lines with their scores
    for _, row in df.iterrows():
        ax[0].text(six_months_data.index[0], row['Price'], f"{row['Score']:.2f}", 
                 color='black', verticalalignment='center', horizontalalignment='left', fontsize=10,
                 bbox=dict(facecolor='white', edgecolor='none', pad=2))

    # Zoom and pan functionality
    def zoom_factory(ax, base_scale=2.):
        def zoom(event):
            cur_xlim = ax.get_xlim()
            cur_ylim = ax.get_ylim()
            xdata = event.xdata  # get event x location
            ydata = event.ydata  # get event y location

            if event.button == 'up':
                # deal with zoom in
                scale_factor = 1 / base_scale
            elif event.button == 'down':
                # deal with zoom out
                scale_factor = base_scale
            else:
                # deal with something that should never happen
                scale_factor = 1
                print(event.button)

            new_width = (cur_xlim[1] - cur_xlim[0]) * scale_factor
            new_height = (cur_ylim[1] - cur_ylim[0]) * scale_factor

            relx = (cur_xlim[1] - xdata) / (cur_xlim[1] - cur_xlim[0])
            rely = (cur_ylim[1] - ydata) / (cur_ylim[1] - cur_ylim[0])

            ax.set_xlim([xdata - new_width * (1 - relx), xdata + new_width * (relx)])
            ax.set_ylim([ydata - new_height * (1 - rely), ydata + new_height * (rely)])
            ax.figure.canvas.draw()

        fig.canvas.mpl_connect('scroll_event', zoom)

    zoom_factory(ax[0])  # Enable zoom on the first axis (candlestick chart)

    plt.show()

# Function to update the charts and table
def update_chart(symbol, end_date):
    global df  # Ensure df is updated globally
    clear_output(wait=True)  # Clear previous output

    # Compute volume profiles (both 6-month and 3-month) for the selected end date
    volume_profile_df, smoothed_volume_profile_six, smoothed_volume_profile_three,raw_volume_profile_six, raw_volume_profile_three, price_bins = compute_volume_profile_with_bins(stock_data, end_date)

    display(volume_profile_df)
    # Append Fibonacci levels to the DataFrame, using only the most recent 6 months
    df = append_fibonacci_levels_to_df(stock_data.loc[stock_data.index <= end_date], volume_profile_df, smoothed_volume_profile_six, price_bins, end_date)

    # Calculate technical indicators for the selected date
    indicators = calculate_technical_indicators(stock_data, df, end_date, PORTFOLIO_SIZE)

    # Display the table of technical indicators
    display_technical_indicators_table(indicators)

    # Plot the candlestick chart
    plot_candlestick_chart(symbol, end_date, df)

    # Plot the volume profiles (6-month and 3-month) with the main support and resistance lines
    plot_volume_profile_with_support_resistance(stock_data, df, smoothed_volume_profile_six, smoothed_volume_profile_three, raw_volume_profile_six, raw_volume_profile_three, price_bins, end_date)

    # Plot the full price chart with the main and minor support/resistance levels and Fibonacci levels
    plot_full_price_chart_with_fibonacci(stock_data, end_date, df)

In [76]:
symbol = "TSLA"
stock_data = yf.download(symbol, period="2y")

# Create a slider for selecting the end date
available_dates = stock_data.index.strftime('%Y-%m-%d').tolist()

date_slider = widgets.SelectionSlider(
    options=available_dates,
    value=available_dates[-1],  # Set the initial value to the most recent date
    description="End Date",
    continuous_update=False,
    layout=widgets.Layout(width='100%')
)

# Output widget to display the charts and table
output = widgets.Output()

# Callback function to update the chart when the slider changes
def on_slider_change(change):
    end_date = change['new']
    with output:
        update_chart(symbol, end_date)

# Update the chart when the slider value changes
date_slider.observe(on_slider_change, names='value')

# Display the slider and the output
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
display(date_slider)
display(output)

# Initialize the chart with the most recent date
on_slider_change({'new': available_dates[-1]})

[*********************100%%**********************]  1 of 1 completed


SelectionSlider(continuous_update=False, description='End Date', index=503, layout=Layout(width='100%'), optio…

Output()

In [75]:
df

,Price,Score,Three-Month Score,Volume Velocity,Label,Is Fibonacci,Volume
0,164.080002,0.515089,NaN,NaN,Fib 0.000,True,166399651.0
1,170.051430,1.000000,0.000000,-1.000000,Minor Support,False,NaN
2,183.487143,0.365828,0.000000,-0.365828,Minor Support,False,NaN
3,190.951428,0.473435,0.000000,-0.473435,Minor Support,False,NaN
4,210.358569,0.730199,0.938351,0.208152,Minor Support,False,NaN
5,222.301426,0.741348,1.000000,0.258652,Main Support,False,NaN
6,237.229996,0.152676,NaN,NaN,Fib 1.000,True,49322167.0
7,254.493394,0.000000,NaN,NaN,Fib 1.236,True,0.0
8,265.173293,0.000000,NaN,NaN,Fib 1.382,True,0.0
